In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
num_vars = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
 "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"]
num_vars_y = num_vars[:]
num_vars_y.append("Cover_Type")

def split_df(df):
    X = df.loc[:, df.columns != 'Cover_Type']
    Y = df.loc[:, df.columns == 'Cover_Type']
    del df
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 101)
    return X_train ,X_test, Y_train, Y_test

def sc_split_df(df):
    X = df.loc[:, df.columns != 'Cover_Type']
    Y = df.loc[:, df.columns == 'Cover_Type']
    del df
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 101)
    del X
    del Y

    X_train, X_test = X_train.copy(), X_test.copy()

    sc = StandardScaler()
    X_train_sc = sc.fit_transform(X_train[num_vars])
    X_test_sc = sc.transform(X_test[num_vars])

    del X_train
    del X_test
    
    return X_train_sc ,X_test_sc, Y_train, Y_test


def forest_test(X_train, X_test, Y_train, Y_test, hyperparams = {
                                                                'bootstrap': False,
                                                                'max_depth': 37,
                                                                'max_features': "auto",
                                                                'min_samples_leaf': 1,
                                                                'min_samples_split': 3,
                                                                'n_estimators': 555
                                                            }):
    hyperparams['n_jobs'] = -2
        
    start = time.process_time()
    
    trainedforest = RandomForestClassifier()
    trainedforest = trainedforest.set_params(**hyperparams)
    trainedforest.fit(X_train,Y_train.values.ravel())
    
    print(time.process_time() - start)
    
    predictionforest = trainedforest.predict(X_test)
    print(confusion_matrix(Y_test,predictionforest))
    print(classification_report(Y_test,predictionforest))
    print("\n")


In [ ]:
import time
from sklearn import tree
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("../input/forest-cover-type-dataset/covtype.csv", sep = ",", header = 0)
print (df.shape)
df.head()

In [ ]:
fig = px.imshow(
    df[num_vars_y].corr()
)

fig.show()

In [ ]:
fig = px.scatter_matrix(
    df,
    dimensions=num_vars,
    color="Cover_Type"
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
X_train_sc ,X_test_sc, Y_train, Y_test = sc_split_df(df[num_vars_y])
X_train_sc ,X_test_sc, Y_train, Y_test = pd.DataFrame(X_train_sc) ,pd.DataFrame(X_test_sc), pd.DataFrame(Y_train), pd.DataFrame(Y_test)

forest_test(X_train_sc ,X_test_sc, Y_train, Y_test)